In [16]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn import metrics

from sklearn.preprocessing import OneHotEncoder

from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, f_regression, f_classif


# Загрузка данных

In [17]:
data = pd.read_excel('data/data_ford_price.xlsx') 

In [18]:
#Выделим данные и целевой признак
y = data['price']
X = data.drop(columns='price')

### Предобработка: заполнение пропусков

In [19]:
#удалим данные типа object
object_columns = [s for s in X.columns if X[s].dtypes == 'object']
X.drop(object_columns, axis = 1, inplace=True)

#определяем модель и заполняем данные по признаку 'weather'
#создаём копию датасета X
data1 = X.copy()
#сохраняем в датасет test_data все пустые значения по выбранному признаку
test_data = data1[data1['weather'].isnull()]
#Удаляем из копии все пропуски
data1.dropna(inplace=True)
#определяем целевой признак
y_train_name = data1['weather']
#выделяем датасеты для определение целевого признака
X_train_name = data1.drop('weather', axis=1)
X_test_name = test_data.drop('weather', axis=1)
#обучаем модель на имеющихся данных
model = LinearRegression()
model.fit(X_train_name, y_train_name)
#делаем предсказание
y_pred = model.predict(X_test_name)
#заполняем пропуски в данных
for i, ni in enumerate(test_data.index[:len(X)]):
        X['weather'].loc[ni] = y_pred[i]


C:\Users\LesenaM\AppData\Local\Temp\ipykernel_17396\882132798.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['weather'].loc[ni] = y_pred[i]


In [20]:
#проверим остались ли пропуски в данных
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   year       7017 non-null   int64  
 1   condition  7017 non-null   int64  
 2   cylinders  7017 non-null   int64  
 3   odometer   7017 non-null   int64  
 4   lat        7017 non-null   float64
 5   long       7017 non-null   float64
 6   weather    7017 non-null   float64
dtypes: float64(3), int64(4)
memory usage: 383.9 KB


###  Отбор признаков

In [21]:
#выделим тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=40)

**Метод рекурсивного исключения признаков**

In [22]:
#определим модель и наиболее значимые признаки для предсказания 'price'
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)

print(F'Наиболее значимые признаки по методу рекурсивного исключения признаков:  {selector.get_feature_names_out()}')

Наиболее значимые признаки по методу рекурсивного исключения признаков:  ['year' 'condition' 'cylinders']


In [23]:
#определяем датасет для обучения модели 
X_train_RFE = X_train[selector.get_feature_names_out()]
X_test_RFE = X_test[selector.get_feature_names_out()]

#обучаем модель на полученных данных
linear_model = LinearRegression()
linear_model.fit(X_train_RFE, y_train)

#делаем предсказание целевого признака
pred_y_RFE = linear_model.predict(X_test_RFE)

#Рассчитываем MAE
print('MAE score: {:.3f} thou. $'.format(metrics.mean_absolute_error(y_test, pred_y_RFE)))
#Рассчитываем MAPE
print('MAPE score: {:.3f} %'.format(metrics.mean_absolute_percentage_error(y_test, pred_y_RFE) * 100))
#Рассчитываем коэффициент детерминации
print('R2 score: {:.3f}'.format(metrics.r2_score(y_test, pred_y_RFE)))


MAE score: 5142.962 thou. $
MAPE score: 16998.376 %
R2 score: 0.446


**МЕТОДЫ ВЫБОРА ПРИЗНАКОВ НА ОСНОВЕ ФИЛЬТРОВ**

In [24]:
selector_SKB = SelectKBest(score_func=f_classif, k=3)
selector_SKB.fit(X_train, y_train)
 
print(F'Наиболее значимые признаки по методу SelectKBest:  {selector_SKB.get_feature_names_out()}')

Наиболее значимые признаки по методу SelectKBest:  ['year' 'condition' 'cylinders']


In [25]:
#определяем датасет для обучения модели 
X_train_SKB = X_train[selector_SKB.get_feature_names_out()]
X_test_SKB = X_test[selector_SKB.get_feature_names_out()]

#обучаем модель на полученных данных
linear_model = LinearRegression()
linear_model.fit(X_train_SKB, y_train)

#делаем предсказание целевого признака
pred_y_SKB = linear_model.predict(X_test_SKB)

#Рассчитываем MAE
print('MAE score: {:.3f} thou. $'.format(metrics.mean_absolute_error(y_test, pred_y_SKB)))
#Рассчитываем MAPE
print('MAPE score: {:.3f} %'.format(metrics.mean_absolute_percentage_error(y_test, pred_y_SKB) * 100))
#Рассчитываем коэффициент детерминации
print('R2 score: {:.3f}'.format(metrics.r2_score(y_test, pred_y_SKB)))


MAE score: 5142.962 thou. $
MAPE score: 16998.376 %
R2 score: 0.446


> По полученным результатам мы видим, что модель, обученная на признаках, отбранных с помощью метода SelectKBest показывает более точные результаты. Хотя в целом модель недообучена и требуется её дальнейшее улучшение. 